In [1]:
import os

In [2]:
%pwd

'c:\\Users\\user\\Krish_Naik_ML\\End-to-end-Machine-Learning-project-with-MLflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\user\\Krish_Naik_ML\\End-to-end-Machine-Learning-project-with-MLflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/appdoh2007/End-to-end-Machine-Learning-project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="appdoh2007"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f76f1a3b23dcdf412ca6ac528bedb975c5c8de007"

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
   root_dir: Path
   test_data_path: Path
   model_path: Path
   all_params: dict
   metric_file_name: Path
   target_column: str
   mlflow_uri: str

In [16]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
   def __init__(
      self,
      config_filepath = CONFIG_FILE_PATH,
      params_filepath = PARAMS_FILE_PATH,
      schema_filepath = SCHEMA_FILE_PATH):
      
      self.config = read_yaml(config_filepath)
      self.params = read_yaml(params_filepath)
      self.schema = read_yaml(schema_filepath)
      
      create_directories([self.config.artifacts_root])
      
   def get_model_evaluation_config(self) -> ModelEvaluationConfig:
      config = self.config.model_evaluation
      params = self.params.ElasticNet
      schema = self.schema.TARGET_COLUMN
      
      create_directories([config.root_dir])
      
      model_evaluation_config = ModelEvaluationConfig(
         root_dir = config.root_dir,
         test_data_path = config.test_data_path,
         model_path = config.model_path,
         all_params = params,
         metric_file_name = config.metric_file_name,
         target_column = schema.name,
         mlflow_uri = "https://dagshub.com/appdoh2007/End-to-end-Machine-Learning-project-with-MLflow.mlflow",
      )
      
      return model_evaluation_config

In [24]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import json
from mlProject.utils.common import save_json

In [25]:
class ModelEvaluation:
   def __init__(self, config: ModelEvaluationConfig):
      self.config = config
      
   def eval_metrics(self, actual, pred):
      rmse = np.sqrt(mean_squared_error(actual, pred))
      mae = mean_absolute_error(actual, pred)
      r2 = r2_score(actual, pred)
      return rmse, mae, r2
   
   
   def log_into_mlflow(self):
      
      test_data = pd.read_csv(self.config.test_data_path)
      model = joblib.load(self.config.model_path)
      
      test_x = test_data.drop([self.config.target_column], axis=1)
      test_y = test_data[[self.config.target_column]]
      
      mlflow.set_tracking_uri(self.config.mlflow_uri)
      tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
      
      
      with mlflow.start_run():
         predicted_qualities = model.predict(test_x)
         
         (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
         
         # Saving metrics as local
         scores = {"rmse": rmse, "mae": mae, "r2": r2}
         save_json(path=Path(self.config.metric_file_name), data=scores)
         
         mlflow.log_param(self.config.all_params)
         
         mlflow.log_metric("rmse", rmse)
         mlflow.log_metric("r2", r2)
         mlflow.log_metric("mae", mae)
         
        # Model registry does not work with file store
         if tracking_uri_type_store != "file":
            
            mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
         else:
            mlflow.sklearn.log_model(model, "model")           

In [26]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.train()
except Exception as e:
    raise e

[2024-02-03 08:07:13,983: INFO: common] yaml file: config\config.yaml loaded successfully
[2024-02-03 08:07:13,984: INFO: common] yaml file: params.yaml loaded successfully
[2024-02-03 08:07:13,985: INFO: common] yaml file: schema.yaml loaded successfully
[2024-02-03 08:07:13,985: INFO: common] created directory at: artifacts


AttributeError: 'ConfigurationManager' object has no attribute 'get_model_evaluation_config'